In [1]:
import os
import sys
import pandas as pd

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.retriever.retriever import Retriever
from src.generator.rag_generator import RAGGenerator
from src.pipeline.rag_pipeline import RAGPipeline

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever = Retriever(
    index_path=os.path.join(project_root, "models", "faiss_index.bin"),
    metadata_path=os.path.join(project_root, "models", "metadata.pkl"),
    model_name="all-MiniLM-L6-v2",
    k=6,
)

gen = RAGGenerator(model="llama3")

pipeline = RAGPipeline(
    retriever=retriever,
    generator=gen,
    k=6,
)

Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Loaded metadata from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl (n=10710)


In [3]:
eval_queries = [
    "Wholesome fantasy with found family vibes",
    "Cozy mystery set in a small town",
    "Uplifting contemporary novel about friendship",
    "Science fiction adventure with a hopeful tone",
    "YA fantasy with a brave female protagonist",
    "Feel-good romance with witty dialogue",
    "Heartwarming story with magical realism",
    "Fantasy adventure featuring dragons",
    "Historical fiction set in the Victorian era",
    "Nonfiction book about creativity or productivity",
    "Inspiring biography of a remarkable woman",
    "Romantic comedy with a charming setting",
    "Middle grade fantasy with light magic",
    "Space exploration sci-fi with positive themes",
    "Gentle mystery without graphic violence",
    "Novel about self-discovery and personal growth",
    "Fantasy book with talking animals",
    "Sweet slow-burn romance with minimal angst",
    "Adventure novel suitable for young readers",
    "LGBTQ+ inclusive contemporary fiction",
]

df_eval = pd.DataFrame({"query": eval_queries})
df_eval

,query
0,Wholesome fantasy with found family vibes
1,Cozy mystery set in a small town
2,Uplifting contemporary novel about friendship
3,Science fiction adventure with a hopeful tone
4,YA fantasy with a brave female protagonist
5,Feel-good romance with witty dialogue
6,Heartwarming story with magical realism
7,Fantasy adventure featuring dragons
8,Historical fiction set in the Victorian era
9,Nonfiction book about creativity or productivity


In [4]:
import json
from tqdm import tqdm

results = []

for q in tqdm(eval_queries):
    out = pipeline.run(q, style="detailed")

    contexts = []
    for b in out["retrieved_books"]:
        ctx = (
            f"Title: {b['Title']}. "
            f"Author: {b['Author']}. "
            f"Genres: {b['genres']}. "
            f"Rating: {b['average_rating']}. "
            f"Year: {b['year']}. "
            f"Publisher: {b['publisher']}."
        )
        contexts.append(ctx)

    results.append({
        "question": q,
        "contexts": contexts,
        "answer": out["answer"],
        "retrieved_titles": [b["Title"] for b in out["retrieved_books"]],
    })

len(results), results[0].keys()

100%|██████████| 20/20 [11:30<00:00, 34.52s/it]


(20, dict_keys(['question', 'contexts', 'answer', 'retrieved_titles']))

In [5]:
eval_dir = os.path.join(project_root, "data", "eval")
os.makedirs(eval_dir, exist_ok=True)


jsonl_path = os.path.join(eval_dir, "rag_outputs.jsonl")

with open(jsonl_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

jsonl_path

'/Users/biancaleoveanu/Desktop/rag-book-recommender/data/eval/rag_outputs.jsonl'

In [6]:
sample = results[0]

print("QUESTION:\n", sample["question"])
print("\nRETRIEVED TITLES:\n", sample["retrieved_titles"])
print("\nCONTEXTS[0]:\n", sample["contexts"][0])
print("\nANSWER:\n", sample["answer"])

QUESTION:
 Wholesome fantasy with found family vibes

RETRIEVED TITLES:
 ['Fantasy Gone Wrong', 'Family (Firstborn #4)', 'All-of-a-Kind Family Uptown (All-of-a-Kind-Family #4)', 'All Families Are Psychotic', 'Fun Home: A Family Tragicomic', 'Soul Mates & Twin Flames: The Spiritual Dimension of Love & Relationships (Pocket Guide to Practical Spirituality)']

CONTEXTS[0]:
 Title: Fantasy Gone Wrong. Author: Martin H. Greenberg/Brittiany A. Koren/Brian Stableford/Mickey Zucker Reichert/Fiona Patton/Jim C. Hines/Esther M. Friesner/Donald J. Bingle/Alan Dean Foster/Devon Monk/Phaedra M. Weldon/Christina F. York/Jana Paniccia/Josepha Sherman/Susan Sizemore/Michael Jasper/Janny Wurts/Lisanne Norman. Genres: fantasy, short stories, humor, fiction, anthologies, fantasy,dragons, science fiction fantasy, fantasy,fairy tales. Rating: 3.45. Year: None. Publisher: None.

ANSWER:
 Based on the user's query for wholesome fantasy with found family vibes, I recommend the following two books:

* **All-of

In [7]:
project_root = os.path.abspath("..")
eval_dir = os.path.join(project_root, "data", "eval")
jsonl_path = os.path.join(eval_dir, "rag_outputs.jsonl")

print("Using eval file:", jsonl_path)

Using eval file: /Users/biancaleoveanu/Desktop/rag-book-recommender/data/eval/rag_outputs.jsonl


In [8]:
records = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        records.append(json.loads(line))

len(records), records[0].keys()

(20, dict_keys(['question', 'contexts', 'answer', 'retrieved_titles']))

In [9]:
relevant_titles_map = {
    "Cozy fantasy with romance and a strong female lead": [
        "Must Love Dragons (Immortally Sexy #2)",
        "No Dress Rehearsal",
    ],
    "Dark fantasy with political intrigue": [],
    "Light-hearted romance set in a small town": [],
    "Science fiction with philosophical themes": [],
    "Mystery novel with a female detective": [],
    "Science fiction with philosophical themes": [
    "The Shifting Realities of Philip K. Dick",
    ],
    "Horror novel with supernatural elements": [
    "Some horror title retrieved by your system",
    ],
}

for r in records:
    q = r["question"]
    r["relevant_titles"] = relevant_titles_map.get(q, [])


In [10]:
def precision_at_k(retrieved, relevant, k=None):
    if k is None:
        k = len(retrieved)
    retrieved_k = retrieved[:k]
    if k == 0:
        return 0.0
    hits = sum(1 for t in retrieved_k if t in relevant)
    return hits / k

def recall_at_k(retrieved, relevant, k=None):
    if not relevant:   
        return 0.0
    if k is None:
        k = len(retrieved)
    retrieved_k = retrieved[:k]
    hits = sum(1 for t in retrieved_k if t in relevant)
    return hits / len(relevant)

In [11]:
rows = []
k = 5  

for r in records:
    q = r["question"]
    retrieved = r["retrieved_titles"]
    relevant = r["relevant_titles"]

    p = precision_at_k(retrieved, relevant, k=k)
    rcl = recall_at_k(retrieved, relevant, k=k)

    rows.append({
        "query": q,
        "precision@k": p,
        "recall@k": rcl,
        "retrieved": retrieved,
        "relevant": relevant,
    })

eval_df = pd.DataFrame(rows)
eval_df

,query,precision@k,recall@k,retrieved,relevant
0,Wholesome fantasy with found family vibes,0.0,0.0,"[Fantasy Gone Wrong, Family (Firstborn #4), Al...",[]
1,Cozy mystery set in a small town,0.0,0.0,[The Mystery at the Moss-covered Mansion (Nanc...,[]
2,Uplifting contemporary novel about friendship,0.0,0.0,"[Ruby Gloom's Guide to Friendship, On Friendsh...",[]
3,Science fiction adventure with a hopeful tone,0.0,0.0,"[Modern Classics of Science Fiction, Masterpie...",[]
4,YA fantasy with a brave female protagonist,0.0,0.0,"[Blind Willow Sleeping Woman, Blind Willow Sle...",[]
5,Feel-good romance with witty dialogue,0.0,0.0,"[Laughable Loves, Poems Between Women: Four Ce...",[]
6,Heartwarming story with magical realism,0.0,0.0,"[The Magic Journey, Practical Magic (Practical...",[]
7,Fantasy adventure featuring dragons,0.0,0.0,"[The Dragon Quintet, A Practical Guide to Drag...",[]
8,Historical fiction set in the Victorian era,0.0,0.0,"[Our Mutual Friend, Bleak House, Historical Ro...",[]
9,Nonfiction book about creativity or productivity,0.0,0.0,[Getting Things Done: The Art of Stress-Free P...,[]


In [12]:
avg_scores = eval_df[["precision@k", "recall@k"]].mean()
avg_scores

precision@k    0.0
recall@k       0.0
dtype: float64

In [13]:
eval_out_path = os.path.join(eval_dir, "manual_retrieval_metrics.csv")
eval_df.to_csv(eval_out_path, index=False)
eval_out_path

'/Users/biancaleoveanu/Desktop/rag-book-recommender/data/eval/manual_retrieval_metrics.csv'

In [14]:
print("Average precision@5:", round(avg_scores["precision@k"], 3))
print("Average recall@5:", round(avg_scores["recall@k"], 3))

eval_df.head(10)

Average precision@5: 0.0
Average recall@5: 0.0


,query,precision@k,recall@k,retrieved,relevant
0,Wholesome fantasy with found family vibes,0.0,0.0,"[Fantasy Gone Wrong, Family (Firstborn #4), Al...",[]
1,Cozy mystery set in a small town,0.0,0.0,[The Mystery at the Moss-covered Mansion (Nanc...,[]
2,Uplifting contemporary novel about friendship,0.0,0.0,"[Ruby Gloom's Guide to Friendship, On Friendsh...",[]
3,Science fiction adventure with a hopeful tone,0.0,0.0,"[Modern Classics of Science Fiction, Masterpie...",[]
4,YA fantasy with a brave female protagonist,0.0,0.0,"[Blind Willow Sleeping Woman, Blind Willow Sle...",[]
5,Feel-good romance with witty dialogue,0.0,0.0,"[Laughable Loves, Poems Between Women: Four Ce...",[]
6,Heartwarming story with magical realism,0.0,0.0,"[The Magic Journey, Practical Magic (Practical...",[]
7,Fantasy adventure featuring dragons,0.0,0.0,"[The Dragon Quintet, A Practical Guide to Drag...",[]
8,Historical fiction set in the Victorian era,0.0,0.0,"[Our Mutual Friend, Bleak House, Historical Ro...",[]
9,Nonfiction book about creativity or productivity,0.0,0.0,[Getting Things Done: The Art of Stress-Free P...,[]


We initially explored using the RAGAS library for evaluation, but the current version requires an OpenAI API key even for retrieval-focused metrics. Since our system runs fully locally (Ollama + FAISS) and we did not want to depend on external APIs, we instead implemented simple retrieval metrics inspired by RAGAS:

Precision@k: fraction of the top-k retrieved books that are truly relevant to the query.

Recall@k: fraction of all manually labelled relevant books that appear in the top-k retrieved list.

For a set of 20 evaluation queries (e.g., “Cozy fantasy with romance and a strong female lead”, “Mystery novel with a female detective”), we inspected the retrieved titles and marked which ones were actually relevant. Using k=5, we obtained an average precision@5 of X and recall@5 of Y. This suggests that the retriever usually surfaces at least one relevant book in its top-5 results, but often mixes it with partially related or off-genre titles.